In [ ]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
plt.rcParams['image.cmap'] = 'gray' # we want our images to be show black and white, not heat-mapped

In [ ]:
session = tf.InteractiveSession()

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Convolutional MNIST model

We want a couple hidden layers not just one and we don't like code duplication

In [ ]:
def convolutional_layer(inp, output_channels, scope_name):
    # We want to get the number of channels in `inp`
    # `get_shape` return a list, describing the static shape of `inp`
    # and the last element of it is the number of channels
    input_channels = inp.get_shape().as_list()[-1]
    # The kernel shape is [kernel_height, kernel_width, input_channels, output_channels]
    kernel_shape = [3 ,3, input_channels, output_channels]
    with tf.variable_scope(scope_name):
        kernel = tf.get_variable('kernel', kernel_shape, initializer=tf.random_normal_initializer(stddev=1e-3))
        bias = tf.get_variable('bias', [output_channels], initializer=tf.zeros_initializer())
    # Strides tells TF the location of the next kernel center relative to the current.
    # Padding tells TF how to behave near the edges of the image 
    # **TODO** add images describing strides and padding
    processed = tf.nn.conv2d(inp, kernel, strides=[1, 2, 2, 1], padding='VALID') + bias
    return tf.nn.leaky_relu(processed)
    

In [ ]:
image = tf.placeholder(tf.float32, [None, 28, 28, 1], name='image')
gt_label = tf.placeholder(tf.float32, [None, 10], name='gt_label')

We now want `image` to be a batch of 28x28 images with one (luminance) channel.

In [ ]:
hidden_layer_1 = convolutional_layer(image, 4, 'conv_1')
hidden_layer_2 = convolutional_layer(hidden_layer_1, 8, 'conv_1')

In [ ]:
hidden_layer_2 = convolutional_layer(hidden_layer_1, 8, 'conv_2')
print(hidden_layer_2.get_shape())

In [ ]:
hidden_layer_3 = convolutional_layer(hidden_layer_2, 16, 'conv_3')
print(hidden_layer_3.get_shape())

In [ ]:
flattened_hidden_layer_3 = tf.reshape(hidden_layer_3, [-1, 2 * 2 * 16])

In [ ]:
logits = tf.layers.dense(flattened_hidden_layer_3, 10, name='final_layer')

In [ ]:
predicted_label_probs = tf.nn.softmax(logits)
predicted_label = tf.argmax(predicted_label_probs, axis=-1)

In [ ]:
loss = tf.nn.softmax_cross_entropy_with_logits_v2(
    labels=gt_label, 
    logits=logits,
)
loss = tf.reduce_mean(loss) # since we want the average loss

In [ ]:
global_step = tf.Variable(0, trainable=False, dtype=tf.int32, name='global_step')
optimizer = tf.train.AdamOptimizer(1e-3)
train_op = optimizer.minimize(loss, global_step=global_step)

In [ ]:
gt_label_index = tf.argmax(gt_label, axis=-1)
gt_match = tf.equal(predicted_label, gt_label_index)
accuracy = tf.reduce_mean(tf.cast(gt_match, tf.float32), axis=0)

In [ ]:
init_op = tf.global_variables_initializer()

In [ ]:
summary_writer = tf.summary.FileWriter('summaries_logdir/')

In [ ]:
summary_writer.add_graph(session.graph)

In [ ]:
tf.summary.scalar('loss', loss)
tf.summary.scalar('accuracy', accuracy)

In [ ]:
session.run(init_op)

In [ ]:
session.run(predicted_label_probs, {image: mnist.train.images[0:1]})

In [ ]:
session.run(predicted_label_probs, {image: mnist.train.images[0:1].reshape([-1, 28, 28, 1])})

In [ ]:
val_summary_writer = tf.summary.FileWriter('summaries_logdir/validation/')

In [ ]:
# Train for one epoch
batch_size = 10
summaries_op = tf.summary.merge_all()
for batch_start in range(0, mnist.train.images.shape[0], batch_size):
    # Get the corresponding batches for images and labels
    image_batch = mnist.train.images[batch_start:batch_start + batch_size].reshape([-1, 28, 28, 1])
    label_batch = mnist.train.labels[batch_start:batch_start + batch_size]
    
    # Execute one step of the model. Note that train_op doesn't have a value, but it **HAS** to be executed
    # in order to train the model
    global_step_value, loss_value, accuracy_value, summaries_value, _ = session.run(
        [global_step, loss, accuracy, summaries_op, train_op],
        {image: image_batch, gt_label: label_batch}
    )
    if global_step_value % 100 == 0:
        # Print the loss and accuracy of the model on the *Trainng* *batch*
        print("{:6}: loss: {}, accuracy:{} ".format(global_step_value, loss_value, accuracy_value))
        summary_writer.add_summary(summaries_value, global_step_value)
    if global_step_value % 1000 == 0:
        # Compute the loss and accuracy on the whole *Validation* set. 
        # Note 0: usually the validation set will be too big to compute validations on the whole set
        # Note 1: there is no `train_op` in the tensors we pass to run, so the model doesn't learn 
        #   anything from the validation set (this would be bad)
        global_step_value, loss_value, accuracy_value, summaries_value = session.run(
            [global_step, loss, accuracy, summaries_op],
            {image: mnist.validation.images.reshape([-1, 28, 28, 1]), 
             gt_label: mnist.validation.labels}
        )
        val_summary_writer.add_summary(summaries_value, global_step_value)
        print("VAL: {:6}: loss: {}, accuracy:{} ".format(global_step_value, loss_value, accuracy_value))


# Performance exploration


Let's check out the examples that are causing problems.

In [ ]:
predicted_label_probs_val, gt_match_val = session.run(
    [predicted_label_probs, gt_match],
    {image: mnist.validation.images.reshape([-1, 28, 28, 1]), gt_label: mnist.validation.labels}
)

In [ ]:
np.sum(~gt_match_val)

In [ ]:
failed_predicted_label_probs_val = predicted_label_probs_val[~gt_match_val]
failed_images_val = mnist.validation.images[~gt_match_val]
failed_labels_val = mnist.validation.labels[~gt_match_val]

In [ ]:
failed_predicted_label_probs_val[0], failed_labels_val[0]

In [ ]:
plt.imshow(failed_images_val[0].reshape([28, 28]))

# Saving our models


In [ ]:
saver = tf.train.Saver()

In [ ]:
saver.save(session, 'saved_model/model.ckpt')

In [ ]:
saver.restore(session, 'saved_model/model.ckpt')